# What happens:
File is created and ready to be uploaded in sharefolder

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import win32com.client as win32
from datetime import date
from IPython.lib.display import Audio

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [5]:
directory = os.path.abspath(os.getcwd()) + "/"
rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0010"

### Make connection with Oracle DB

In [6]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Save SQL

In [7]:
%%time
d = pd.read_sql_query("""select r.reporting_date,sp.security_reference as "ISIN", r.counterparty_code as issuer_CCN, r.counterparty_name as issuer_name, r.gcc_head_code as issuer_gcc_CCN, r.gcc_head_name as issuer_gcc_name, 	 
r.risk_country as issuer_country_of_risk, r.maturity_date, sum (r.rwa) as rwa, sum (r.ead_pre_ccf) as exposure_Eur,sum(sp.position_initial_value) as exposure_in_orig_currency, sum(sp.accruals) as accruals_in_orig_currency, s.currency_code as original_currency, c.exch_rate as current_exchange_rate,	 
r.profit_center as CC_Department_in_charge, r.account_manager as relationship_manager,r.rating_code as external_rating, r.internal_rating, r.pd as PD, r.lgd as LGD, r.industry_code_1 as Industry_main,	 
r.industry_code_2 as Industry_detail, r.nace_description as Onace_description, r.contract_type as GPC_Code, r.contract_type_desc as GPC_Desc, r.record_source	 
from rzb_cdr r	 
left join security_positions sp on sp.contract_reference = r.security_positions_ref	 
left join security s on s.security_reference = r.contract_reference	 
left join currency c on c.ccy_code = s.currency_code	 
where r.book_code = 'HO_B'	 
and r.table_name = 'BOND'	 
and counterparty_type in ('3', '6')	 
and s.security_family = 'B'	 
group by (r.reporting_date, r.counterparty_code, r.counterparty_name,r.gcc_head_code, r.gcc_head_name, r.risk_country,sp.security_reference, r.account_manager,r.rating_code, r.internal_rating, s.currency_code, c.exch_rate,	 
r.industry_code_1, r.industry_code_2,r.nace_description,r.profit_center, r.contract_type, r.contract_type_desc,r.pd,r.lgd, r.record_source, r.maturity_date)""", connection_RAY)

CPU times: total: 0 ns
Wall time: 363 ms


### Save var in an excel file

In [8]:
date = rd[:4] + "_" + rd[4:6] + "_" + rd[6:8]  
ex_folder = "Export/"

# determining the name of the file
file_name = ex_folder + date + '_Corporate Bonds HO_B ' + rd + '.xlsx'

# saving the excel
d.to_excel(file_name, index=False)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
